In [ ]:
!pip install torch torchvision matplotlib tensorboard

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276))
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)


100%|██████████| 169001437/169001437 [00:02<00:00, 62707024.52it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
from torchvision.models import vgg16, resnet18

# 加载预训练模型
vgg_model = vgg16(pretrained=True)
resnet_model = resnet18(pretrained=True)

# 如果使用CIFAR-100数据集，需要修改最后的全连接层
vgg_model.classifier[6] = nn.Linear(4096, 100)
resnet_model.fc = nn.Linear(512, 100)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 109MB/s] 
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future.

In [ ]:
import time
vgg16 = torchvision.models.vgg16(pretrained=True)
resnet18 = torchvision.models.resnet18(pretrained=True)

# 修改VGG16最后的全连接层，以适应CIFAR-100（100个分类）
vgg16.classifier[6] = nn.Linear(4096, 100)

# 修改ResNet18的最后全连接层
resnet18.fc = nn.Linear(512, 100)

# 将模型移至GPU（如果有GPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16.to(device)
resnet18.to(device)

# 3. 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()

vgg16_optimizer = optim.SGD(vgg16.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
resnet18_optimizer = optim.SGD(resnet18.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# 学习率调度器
vgg16_scheduler = optim.lr_scheduler.StepLR(vgg16_optimizer, step_size=30, gamma=0.1)
resnet18_scheduler = optim.lr_scheduler.StepLR(resnet18_optimizer, step_size=30, gamma=0.1)

# 4. 定义训练函数
def train_model(model, trainloader, optimizer, criterion, device, scheduler, num_epochs=10):
    model.train()
    writer = SummaryWriter()  # 使用TensorBoard记录训练信息
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}, Accuracy: {accuracy}%')

        # 使用TensorBoard记录训练过程
        writer.add_scalar('training loss', running_loss / len(trainloader), epoch)
        writer.add_scalar('accuracy', accuracy, epoch)

        scheduler.step()

    print('训练结束')
    writer.close()

# 5. 定义评估函数
def evaluate_model(model, testloader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    start_time = time.time()
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    end_time = time.time()
    accuracy = 100 * correct / total
    print(f'Test Loss: {running_loss/len(testloader)}, Test Accuracy: {accuracy}%')
    print(f'Inference Time: {end_time - start_time:.2f} seconds')

# 6. 训练与评估VGG16
print("开始训练VGG16")
train_model(vgg16, trainloader, vgg16_optimizer, criterion, device, vgg16_scheduler, num_epochs=10)
evaluate_model(vgg16, testloader, criterion, device)

# 7. 训练与评估ResNet18
print("开始训练ResNet18")
train_model(resnet18, trainloader, resnet18_optimizer, criterion, device, resnet18_scheduler, num_epochs=10)
evaluate_model(resnet18, testloader, criterion, device)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
 

开始训练VGG16
